In [ ]:
!pip install h5py
!pip install typing-extensions
!pip install wheel

In [ ]:
!pip install docplex


In [ ]:
!pip install --upgrade azure-quantum\[qiskit\]
!pip install qiskit
!pip install cplex

In [ ]:
!pip install openqaoa-qiskit

In [20]:
%matplotlib notebook

# Import external libraries to present an manipulate the data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Import docplex model to generate the problem to optimize
from docplex.mp.model import Model

# Import the libraries needed to employ the QAOA quantum algorithm using OpenQAOA
from openqaoa import QAOA

# method to covnert a docplex model to a qubo problem
from openqaoa.problems.converters import FromDocplex2IsingModel
from openqaoa.backends import create_device

# method to find the corrects states for the QAOA boject
from openqaoa.utilities import ground_state_hamiltonian

## 1) Tic Tac Toe Game
This problem can be solved by treating each cell as a variable, and our objective function aims to maximize the number of winning moves. For instance, if we want to address the following problem:



<div>
<p style = 'text-align:center;'>
<img src="../Images/image10.png" alt="JuveYell" width="300px">
</p>
</div>


So, for x_0, we observe it has 2 winning possibilities. On the other hand, x_1 has only one, x_2 has 0 winning possibilities, and x_3 has only 1 winning option. Therefore, our objective function is formulated as follows:
$$f_1(\textbf{x}) = 2*x_0 + 1*x_1 + 0*x_2 + 1*x_3$$

This objective function is subject to an equality constraint, as when the game starts with 0's, there should be a total of 4 X's played. This forms our constraint, but in this case, given an initial configuration, we subtract 2 X's:

$$x_0 + x_1 + x_2 + x_3 = 2$$



In [95]:
#Specific the Model and put a name
mdl = Model("Basic Problem using maximize")
# Number of variables
n_vars = 4

# Using binary variables for this model
x = mdl.binary_var_list(n_vars, name="x")
obj_func = 2*x[0] + x[1] + 0*x[2] + 1*x[3]
#obj_func = 2*x[0] + x[1] + 1*x[2] + 0*x[3] + 1*x[4]
mdl.maximize(obj_func)
mdl.add_constraint(x[0] + x[1] + x[2] + x[3] == 2)

print(mdl.prettyprint())
# Converting the Docplex model into its qubo representation
qubo = FromDocplex2IsingModel(mdl)

# Ising encoding of the QUBO problem
ising_encoding = qubo.ising_model

# Print in a df the ising encoding (we need to remove keys: 'problem_instance' and 'metadata'))
ising_encoding_dict = ising_encoding.asdict(exclude_keys=['problem_instance', 'metadata'])
#pd.DataFrame(ising_encoding_dict)
#Specific local device usign qiskit backend
device = create_device("local", 'qiskit.qasm_simulator')
#Is possible check the devices using qaoa.local_simulators, qaoa.cloud_provider
qaoa = QAOA(device)

#Indicate the properties to the QAOA quantum algorithm,shots,seed
qaoa.set_backend_properties(n_shots=20, seed_simulator=1)
#check the p value and the variational init params
qaoa.set_circuit_properties(p=2, init_type="custom", variational_params_dict={"betas":2*[0.01*np.pi],"gammas":2*[0.01*np.pi]})

#Indicate the ising e ncoding model from docplex
qaoa.compile(ising_encoding)

#Optimize the quantum algorithm
qaoa.optimize()
#Print in a df the best 10 solutions
results = pd.DataFrame(qaoa.result.lowest_cost_bitstrings(10))
results = results[results["bitstrings_energies"]<0]
results

// This file has been generated by DOcplex
// model name is: Basic Problem using minimize
// var contrainer section
dvar bool x[4];

maximize
 2 x_0 + x_1 + x_3;
 
subject to {
 x_0 + x_1 + x_2 + x_3 == 2;

}
None


,solutions_bitstrings,bitstrings_energies,probabilities
0,1001,-3.0,0.10
1,1100,-3.0,0.10
2,0101,-2.0,0.05
3,1010,-2.0,0.10
4,0110,-1.0,0.10
5,0011,-1.0,0.10



From the results, we observe that for the lowest bitstrings_energy (-3), we have the states 1001 and 1100. These happen to be the only 2 options where X's can win in this case.


Additionally, we can also solve the bonus problem.

<div>
<p style = 'text-align:center;'>
<img src="../Images/image11.png" alt="JuveYell" width="300px">
</p>
</div>

So, for x_0, we observe it has 2 winning possibilities. On the other hand, x_1 has only one, x_2 has only one too, x_3 has 0 winning possibilities, and x_4 has only 1 winning option. Therefore, our objective function is formulated as follows:
$$f_1(\textbf{x}) = 2*x_0 + 1*x_1 + 1*x_2 + 0*x_3 + 1*x_4$$

This objective function is subject to an equality constraint, as when the game starts with 0's, there should be a total of 4 X's played. This forms our constraint, but in this case, given an initial configuration, we subtract 2 X's:

$$x_0 + x_1 + x_2 + x_3 + x_4= 2$$

In [94]:
#Specific the Model and put a name
mdl = Model("Basic Problem using minimize")
# Number of variables
n_vars = 5

# Using binary variables for this model
x = mdl.binary_var_list(n_vars, name="x")
obj_func = 2*x[0] + x[1] + 1*x[2] + 0*x[3] + 1*x[4]

mdl.maximize(obj_func)
mdl.add_constraint(x[0] + x[1] + x[2] + x[3] + x[4] == 2)
print(mdl.prettyprint())
# Converting the Docplex model into its qubo representation
qubo = FromDocplex2IsingModel(mdl)

# Ising encoding of the QUBO problem
ising_encoding = qubo.ising_model

# Print in a df the ising encoding (we need to remove keys: 'problem_instance' and 'metadata'))
ising_encoding_dict = ising_encoding.asdict(exclude_keys=['problem_instance', 'metadata'])

#Specific local device usign qiskit backend
device = create_device("local", 'qiskit.qasm_simulator')
#Is possible check the devices using qaoa.local_simulators, qaoa.cloud_provider
qaoa = QAOA(device)

#Indicate the properties to the QAOA quantum algorithm,shots,seed
qaoa.set_backend_properties(n_shots=20, seed_simulator=1)
#check the p value and the variational init params
qaoa.set_circuit_properties(p=2, init_type="custom", variational_params_dict={"betas":2*[0.01*np.pi],"gammas":2*[0.01*np.pi]})

#Indicate the ising e ncoding model from docplex
qaoa.compile(ising_encoding)

#Optimize the quantum algorithm
qaoa.optimize()
#Print in a df the best 10 solutions
results = pd.DataFrame(qaoa.result.lowest_cost_bitstrings(10))
results = results[results["bitstrings_energies"]<0]
results

// This file has been generated by DOcplex
// model name is: Basic Problem using minimize
// var contrainer section
dvar bool x[5];

maximize
 2 x_0 + x_1 + x_2 + x_4;
 
subject to {
 x_0 + x_1 + x_2 + x_3 + x_4 == 2;

}
None


,solutions_bitstrings,bitstrings_energies,probabilities
0,10001,-3.0,0.05
1,10100,-3.0,0.05
2,11000,-3.0,0.20
3,10010,-2.0,0.10
4,00101,-2.0,0.10
5,01100,-2.0,0.15
6,00011,-1.0,0.05
7,00110,-1.0,0.10
8,01010,-1.0,0.15
